# **Sale Data Analyzing:**

## Importing Libraries:

In [134]:
import pandas as pd
import os

## Reading data and Merging:

In [143]:
#saving path to a variable:
path='./Sales_Data/'

#save files names:
files=(file for file in os.listdir(path) if file.endswith('.csv'))

#creating an empty data frame:
All_data=pd.DataFrame()

#read all files in a single data frame(prevents from  duplicate headers)
for i,file in enumerate(files):
  df=pd.read_csv(path+file)
  duplicate_header=(df==df.iloc[0]).all(axis=1)
  df=df[~duplicate_header]
  if i>0:
    df=df[1:]
    All_data=pd.concat([All_data,df])
  else:
    All_data=df

#save to new csv file
All_data.to_csv('All_data.csv',index=False)





## Reading from new file and Cleaning Data

In [136]:
df=pd.read_csv('All_data.csv')


### Dealing with NaN values:

In [137]:
df.isna().sum()
df.dropna(how='all',inplace=True)
df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
5,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001"
...,...,...,...,...,...,...
186834,259353,AAA Batteries (4-pack),3,2.99,09/17/19 20:56,"840 Highland St, Los Angeles, CA 90001"
186835,259354,iPhone,1,700,09/01/19 16:00,"216 Dogwood St, San Francisco, CA 94016"
186836,259355,iPhone,1,700,09/23/19 07:39,"220 12th St, San Francisco, CA 94016"
186837,259356,34in Ultrawide Monitor,1,379.99,09/19/19 17:30,"511 Forest St, San Francisco, CA 94016"


In [138]:
df.isna().sum()

Order ID            0
Product             0
Quantity Ordered    0
Price Each          0
Order Date          0
Purchase Address    0
dtype: int64

### checking for duplicate values:

In [139]:
df.duplicated().sum()

618

In [140]:
df.drop_duplicates(inplace=True)
df


,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
5,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001"
...,...,...,...,...,...,...
186834,259353,AAA Batteries (4-pack),3,2.99,09/17/19 20:56,"840 Highland St, Los Angeles, CA 90001"
186835,259354,iPhone,1,700,09/01/19 16:00,"216 Dogwood St, San Francisco, CA 94016"
186836,259355,iPhone,1,700,09/23/19 07:39,"220 12th St, San Francisco, CA 94016"
186837,259356,34in Ultrawide Monitor,1,379.99,09/19/19 17:30,"511 Forest St, San Francisco, CA 94016"
